In [8]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from scipy.stats import chisquare
from scipy.stats import chi2
from scipy.stats import poisson

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import chart_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
! pip install plotly

     |████████████████████████████████| 20.6 MB 3.9 MB/s eta 0:00:01


In [2]:
try:
    os.remove("btc.csv")
except FileNotFoundError:
    pass
!wget https://coinmetrics.io/newdata/btc.csv

--2021-06-21 20:12:33--  https://coinmetrics.io/newdata/btc.csv
Resolving coinmetrics.io (coinmetrics.io)... 2606:4700:20::681a:f42, 2606:4700:20::ac43:4bae, 2606:4700:20::681a:e42, ...
Connecting to coinmetrics.io (coinmetrics.io)|2606:4700:20::681a:f42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8511253 (8.1M) [application/octet-stream]
Saving to: ‘btc.csv’

btc.csv             100%[===================>]   8.12M  4.31MB/s    in 1.9s    

2021-06-21 20:12:36 (4.31 MB/s) - ‘btc.csv’ saved [8511253/8511253]



In [3]:
data = pd.read_csv("btc.csv").dropna()
data.columns.values

array(['date', 'AdrActCnt', 'AdrBal1in100KCnt', 'AdrBal1in100MCnt',
       'AdrBal1in10BCnt', 'AdrBal1in10KCnt', 'AdrBal1in10MCnt',
       'AdrBal1in1BCnt', 'AdrBal1in1KCnt', 'AdrBal1in1MCnt', 'AdrBalCnt',
       'AdrBalNtv0.001Cnt', 'AdrBalNtv0.01Cnt', 'AdrBalNtv0.1Cnt',
       'AdrBalNtv100Cnt', 'AdrBalNtv100KCnt', 'AdrBalNtv10Cnt',
       'AdrBalNtv10KCnt', 'AdrBalNtv1Cnt', 'AdrBalNtv1KCnt',
       'AdrBalNtv1MCnt', 'AdrBalUSD100Cnt', 'AdrBalUSD100KCnt',
       'AdrBalUSD10Cnt', 'AdrBalUSD10KCnt', 'AdrBalUSD10MCnt',
       'AdrBalUSD1Cnt', 'AdrBalUSD1KCnt', 'AdrBalUSD1MCnt',
       'AssetEODCompletionTime', 'BlkCnt', 'BlkSizeMeanByte',
       'BlkWghtMean', 'BlkWghtTot', 'CapAct1yrUSD', 'CapMVRVCur',
       'CapMVRVFF', 'CapMrktCurUSD', 'CapMrktFFUSD', 'CapRealUSD',
       'DiffLast', 'DiffMean', 'FeeByteMeanNtv', 'FeeMeanNtv',
       'FeeMeanUSD', 'FeeMedNtv', 'FeeMedUSD', 'FeeTotNtv', 'FeeTotUSD',
       'FlowInExNtv', 'FlowInExUSD', 'FlowOutExNtv', 'FlowOutExUSD',
       'FlowTfr

In [4]:
data['BlkCnt']

926     170
927     142
928     149
929     152
930     151
       ... 
4547    123
4548    130
4549    124
4550    103
4551    105
Name: BlkCnt, Length: 3626, dtype: int64

In [20]:
alpha = 0.025 # Free alpha

data['HashRateL7DInc'] = data['HashRate'].rolling(7).mean()
data['HashRateL7D'] = data['HashRateL7DInc'].shift()
data['HashRateATH'] = data['HashRateL7D'].max()

data['BlkCntLower'] = [poisson.interval(1 - alpha, x)[0] for x in data['BlkCnt']]
data['BlkCntUpper'] = [poisson.interval(1 - alpha, x)[1] for x in data['BlkCnt']]
data['HashRateLower'] = [(x / 144) * y * (((2 ** 32) / (10 **12)) / 600)
                         for x, y in zip(data['BlkCntLower'], data['DiffMean'])]
data['HashRateUpper'] = [(x / 144) * y * (((2 ** 32) / (10 **12)) / 600)
                         for x, y in zip(data['BlkCntUpper'], data['DiffMean'])]

data['HashRateLowerPct'] = (data['HashRateLower'] / data['HashRate']) - 1
data['HashRateUpperPct'] = (data['HashRateUpper'] / data['HashRate']) - 1


data['BlkCntLower1Side'] = [poisson.interval(1 - alpha * 2, x)[0] for x in data['BlkCnt']]
data['BlkCntUpper1Side'] = [poisson.interval(1 - alpha * 2, x)[1] for x in data['BlkCnt']]
data['HashRateLower1Side'] = [(x / 144) * y * (((2 ** 32) / (10 **12)) / 600)
                         for x, y in zip(data['BlkCntLower1Side'], data['DiffMean'])]
data['HashRateUpper1Side'] = [(x / 144) * y * (((2 ** 32) / (10 **12)) / 600)
                         for x, y in zip(data['BlkCntUpper1Side'], data['DiffMean'])]

data['HashRateLower1Side'] = [x if z > y else z
                              for x, y, z in zip(data['HashRateLower1Side'], data['HashRateL7D'], data['HashRate'])]
data['HashRateUpper1Side'] = [x if z < y else z
                              for x, y, z in zip(data['HashRateUpper1Side'], data['HashRateL7D'], data['HashRate'])]

data['HashRateLower1SideDelta'] = ((data['HashRateLower1Side'] / data['HashRateL7D']) - 1) * 1
data['HashRateUpper1SideDelta'] = ((data['HashRateUpper1Side'] / data['HashRateL7D']) - 1) * 1

data['HashRateLower1SideDeltaATH'] = ((data['HashRateLower1Side'] / data['HashRateATH']) - 1) * 1
data['HashRateUpper1SideDeltaATH'] = ((data['HashRateUpper1Side'] / data['HashRateATH']) - 1) * 1

In [27]:
subset = data.loc[
#     (data['date'] >= '2021-03-01')
    (data['date'] == '2021-04-16')
    | (data['date'] == '2021-04-17')
    | (data['date'] == '2021-05-18')
    | (data['date'] == '2021-06-11')
    | (data['date'] == '2021-05-18')
    | (data['date'] == '2021-06-19')
    | (data['date'] == '2021-06-20')
#     & (data['date'] <= '2021-04-17')
][
 ['date', 'HashRate', 'HashRateL7D', 'HashRateLower1Side', 'HashRateUpper1Side', 
#      'HashRateLower1SideDelta', 'HashRateUpper1SideDelta', 
  'HashRateLower1SideDeltaATH', 'HashRateUpper1SideDeltaATH']
#   , 'HashRateLowerPct', 'HashRateUpperPct']
]#.style.format("{:.2%}")

In [28]:
subset.style.format({
    'HashRate': '{:.5E}',
    'HashRateL7D': '{:.5E}',
    'HashRateLower1Side': '{:.5E}',
    'HashRateUpper1Side': '{:.5E}',
#     'HashRateLower1SideDelta': '{:,.2%}',
#     'HashRateUpper1SideDelta': '{:,.2%}',
    'HashRateLower1SideDeltaATH': '{:,.2%}',
    'HashRateUpper1SideDeltaATH': '{:,.2%}',
#     'HashRateLower': '{:,.2%}',
#     'HashRateUpper': '{:,.2%}',
})

,date,HashRate,HashRateL7D,HashRateLower1Side,HashRateUpper1Side,HashRateLower1SideDeltaATH,HashRateUpper1SideDeltaATH
4486,2021-04-16,1.20743E+08,1.72146E+08,1.20743E+08,1.44189E+08,-33.23%,-20.26%
4487,2021-04-17,1.06676E+08,1.64420E+08,1.06676E+08,1.28949E+08,-41.01%,-28.69%
4518,2021-05-18,1.24507E+08,1.67772E+08,1.24507E+08,1.49408E+08,-31.14%,-17.37%
4542,2021-06-11,1.09860E+08,1.40203E+08,1.09860E+08,1.31832E+08,-39.24%,-27.09%
4550,2021-06-19,1.02059E+08,1.30240E+08,1.02059E+08,1.21876E+08,-43.56%,-32.60%
4551,2021-06-20,1.04041E+08,1.25090E+08,1.04041E+08,1.24849E+08,-42.46%,-30.96%


In [32]:
# data
# data.style.format({
#     'HashRateLower1SideDelta': '{:,.2f}',
#     'HashRateUpper1SideDelta': '{:,.2f}',
# })
data[
    ['date', 'HashRate', 'HashRateLower1Side', 'HashRateUpper1Side', 
     'HashRateLower1SideDelta', 'HashRateUpper1SideDelta']][-50:]

,date,HashRate,HashRateLower1Side,HashRateUpper1Side,HashRateLower1SideDelta,HashRateUpper1SideDelta
4493,2021-04-23,1.582560e+08,1.324661e+08,1.582560e+08,0.005083,2.007624e-01
4494,2021-04-24,1.699787e+08,1.430165e+08,1.699787e+08,0.042735,2.393162e-01
4495,2021-04-25,1.652896e+08,1.383275e+08,1.652896e+08,-0.053837,1.305842e-01
4496,2021-04-26,1.336384e+08,1.336384e+08,1.582560e+08,-0.103371,6.179775e-02
4497,2021-04-27,1.664619e+08,1.394997e+08,1.664619e+08,-0.081588,9.592061e-02
4498,2021-04-28,1.664619e+08,1.394997e+08,1.664619e+08,-0.095548,7.926167e-02
4499,2021-04-29,1.430165e+08,1.430165e+08,1.688064e+08,-0.112266,4.781705e-02
4500,2021-04-30,1.699787e+08,1.430165e+08,1.699787e+08,-0.092455,7.863974e-02
4501,2021-05-01,1.582156e+08,1.582156e+08,1.855334e+08,-0.006563,1.649660e-01
4502,2021-05-02,1.895272e+08,1.628909e+08,1.895272e+08,0.033701,2.027335e-01


In [33]:
import plotly.graph_objs as go

temp = data.loc[
    data['date'] >= '2021-01-01'
]


fig = go.Figure([
    go.Scatter(
        x=temp['date'],
        y=temp['HashRate'],
        line=dict(color='darkorange'),
        mode='lines',
        name='Hash Rate (Day)'
    ),
    go.Scatter(
        x=temp['date'],
        y=temp['HashRateL7D'],
        line=dict(color='yellow'),
        mode='lines',
        name='Hash Rate (7DMA)'
    ),
    go.Scatter(
        name='Upper Bound',
        x=temp['date'],
        y=temp['HashRateUpper'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=temp['date'],
        y=temp['HashRateLower'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(190, 190, 190, 0.3)',
        fill='tonexty',
        showlegend=False
    )
])
fig.update_layout(
        annotations=[
            dict(x=1, y=-0.1,
                 text="Chart by: @typerbole; Data: Coinmetrics",
                 showarrow=False, xref='paper', yref='paper',
                 xanchor='right', yanchor='auto', xshift=0, yshift=0)
        ],
        showlegend=True,
        legend_orientation="h",
        template="plotly_dark"

)
fig.update_layout(
        title_text='Bitcoin Hash Rate with 95% Poisson Confidence Bounds'
)
fig.update_yaxes(
        title_text='Hash Rate'
)
fig.show()

In [34]:
import plotly.graph_objs as go

temp = data.loc[
    data['date'] >= '2021-01-01'
]


fig = go.Figure([
    go.Scatter(
        x=temp['date'],
        y=temp['HashRate'],
        line=dict(color='darkorange'),
        mode='lines',
        name='Hash Rate (Day)'
    ),
    go.Scatter(
        x=temp['date'],
        y=temp['HashRateL7D'],
        line=dict(color='yellow'),
        mode='lines',
        name='Hash Rate (7DMA)'
    ),
    go.Scatter(
        name='Upper Bound',
        x=temp['date'],
        y=temp['HashRateUpper1Side'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=temp['date'],
        y=temp['HashRateLower1Side'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(190, 190, 190, 0.3)',
        fill='tonexty',
        showlegend=False
    )
])
fig.update_layout(
        annotations=[
            dict(x=1, y=-0.1,
                 text="Chart by: @typerbole; Data: Coinmetrics",
                 showarrow=False, xref='paper', yref='paper',
                 xanchor='right', yanchor='auto', xshift=0, yshift=0)
        ],
        showlegend=True,
        legend_orientation="h",
        template="plotly_dark"

)
fig.update_layout(
        title_text='Bitcoin Hash Rate with 95% Poisson Confidence Bounds'
)
fig.update_yaxes(
        title_text='Hash Rate'
)
fig.show()